In [1]:
import h5py    
import numpy as np    
import pandas as pd
from astropy.cosmology import WMAP9 as cosmo
from astropy.cosmology import z_at_value
import astropy.units as u
from astropy import constants as const


In [2]:
yrsec = (1*u.yr).to(u.s).value # 1 year in secs
solar_mass = const.M_sun.value #in Kgs
#hubble_time = (1/cosmo.H(0)).to(u.yr).value #Hubble time = 1/H0 
age_of_the_universe = cosmo.age(0).to(u.yr).value

In [3]:
#parent folder
pfolder = "/Users/pranavsatheesh/Triples/Github/"
import sys
sys.path.append(pfolder)

In [4]:
#binary merger files 
file_name = pfolder + "Illustris_Data/mbhb-evolution_no-ecc_lc-full-0.6.hdf5"
f1 = h5py.File(file_name,'r') 

In [19]:
for grp in f1.keys():
    print("grp: ",grp)
    print(f1[grp].name)
    for a in f1[grp].attrs.keys():
        print(a)
    for k in f1[grp].keys():
        print("key: ", k)
        for a in f1[grp][k].attrs:
            print(a,": ",f1[grp][k].attrs[a])


grp:  evolution
/evolution
key:  dadt
desc :  Hardening rate of each binary at each timestep  Shape: binaries, steps.  Units: [cm/s]
shape :  binaries, steps
units :  cm/s
key:  dadt_df
desc :  Hardening rate due to dynamical friction of each binary at each timestep  Shape: binaries, steps.  Units: [cm/s]
shape :  binaries, steps
units :  cm/s
key:  dadt_gw
desc :  Hardening rate due to gravitational waves of each binary at each timestep  Shape: binaries, steps.  Units: [cm/s]
shape :  binaries, steps
units :  cm/s
key:  dadt_lc
desc :  Hardening rate due to stellar-scattering of each binary at each timestep  Shape: binaries, steps.  Units: [cm/s]
shape :  binaries, steps
units :  cm/s
key:  dadt_vd
desc :  Hardening rate due to disk torque of each binary at each timestep  Shape: binaries, steps.  Units: [cm/s]
shape :  binaries, steps
units :  cm/s
key:  dedt
desc :  Eccentricity evolution rate of each binary at each timestep  Shape: binaries, steps.  Units: [1/s]
shape :  binaries, s

In [10]:
#binary merger files 
file_name = pfolder + "Illustris_Data/mbhb-evolution_no-ecc_lc-full-0.6.hdf5"
f1 = h5py.File(file_name,'r') 

Ms = np.array(f1['evolution']['masses'])
t = np.array(f1['evolution']['times'])
r = np.array(f1['evolution']['sep'])

Ms = Ms/(solar_mass*10**3)
Nbinary = len(Ms)
M1 = Ms[:,0]
M2 = Ms[:,1]

In [11]:
# the merger is the maximum value in each time evolution of the binary

t_bin_merger = np.amax(t,1)/yrsec
merger = [] #to indicate if the binary actually merges before hubble time
z_binary = [] #convert the time of merger to redshifts

In [12]:
for time in t_bin_merger:
    if time >= age_of_the_universe:
        #these black holes aren't merging
        merger.append("No")
        z_binary.append(0)
    else:
        merger.append("Yes")
        z_binary.append(z_at_value(cosmo.age,(time/10**9)*u.Gyr,zmin=1e-9).value)

In [13]:
df = pd.DataFrame([M1,M2,t_bin_merger,merger,z_binary])
df = df.transpose()
df.columns = ['M1', 'M2', 't_merger','Merger','Redshift']

In [14]:
df.head()

,M1,M2,t_merger,Merger,Redshift
0,2261178.977273,1629005.681818,858774126.450153,Yes,6.459185
1,1614772.727273,1077349.431818,893400572.050411,Yes,6.265176
2,6554857.954545,1175723.011364,4120229485.36754,Yes,1.587945
3,7039616.477273,6263607.954545,89337138643.547943,No,0
4,11791392.045455,3306548.295455,14217143975.611784,No,0


In [15]:

#binary ids
mergers = np.load(pfolder+'Illustris_Data/ill-1_blackhole_mergers_fixed.npz')
indexes = f1['evolution']['val_inds'][:]
binary_ids = mergers['ids'][indexes]

In [16]:
Triple_df = pd.read_csv(pfolder+"Illustris_Data/triple_data_ill.csv") #the triples data file from find_triples
Triple_df.head()

,bhid1_bin_1,bhid2_bin_1,bhid1_bin_2,bhid2_bin_2,M1_bin_1,M2_bin_1,M1_bin_2,M2_bin_2,t1,t2,tmerger
0,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,3.286581e+07,1.253781e+07,5.276829e+07,8.204738e+06,2.055999e+09,1.988185e+09,1.985633e+09
1,9.223372e+18,9.223372e+18,3.402509e+18,9.223372e+18,1.059404e+07,2.885124e+06,3.132749e+08,4.211761e+07,3.255727e+11,7.265512e+09,5.529691e+09
2,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,1.655162e+07,6.537747e+06,7.257406e+07,4.653945e+07,5.274952e+10,1.032617e+10,3.025313e+09
3,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,1.259004e+07,3.439988e+06,2.386990e+07,2.216544e+06,5.883422e+09,1.284130e+12,1.972738e+09
4,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,1.325181e+08,1.408275e+07,4.551391e+08,1.002870e+06,1.206497e+10,7.389760e+09,4.814316e+09


In [17]:
np.max(Triple_df["M1_bin_1"])

13255795998.36437

In [18]:
bh1id1 = Triple_df["bhid1_bin_1"].to_numpy()
bh1id2 = Triple_df["bhid2_bin_1"].to_numpy()
bh2id1 = Triple_df["bhid1_bin_2"].to_numpy()
bh2id2 = Triple_df["bhid2_bin_2"].to_numpy()

In [131]:
def searchID(id):

    #to search if the binary ids belong to any triples, indicating a triple interaction

    tf1 = id in bh1id1
    tf2 = id in bh1id2
    tf3 = id in bh2id1
    tf4 = id in bh2id2

    tf = tf1 + tf2 + tf3 + tf4

    return tf 

merger_type = []

In [132]:
indices = []
for i in range(Nbinary):

    search1 = searchID(binary_ids[i][0])
    search2 = searchID(binary_ids[i][1])

    search = search1 + search2
    if(search == 0):
        merger_type.append("iso")
        
    else:
        merger_type.append("trip")
        indices.append(i)
       

df["Type"] = merger_type


In [133]:

binary_ids = np.array(binary_ids)
df["BH1-ID"] = binary_ids[:,0]
df["BH2-ID"] = binary_ids[:,1]

print("Number of Binaries: %d"%(len(df)))
print("Number of merged binaries: %d"%(len(df[df["Merger"]=="Yes"])))
print("Number of Isolated Binary mergers: %d"%(len(df[(df["Merger"]=="Yes") & (df["Type"] == "iso")])))
print("Number of Binaries with triple influence: %d"%(len(df[(df["Type"] == "trip")])))
print("Number of Binary non mergers with triple influence: %d"%(len(df[(df["Merger"]=="No") & (df["Type"] == "trip")])))

Number of Binaries: 9234
Number of merged binaries: 2370
Number of Isolated Binary mergers: 1929
Number of Binaries with triple influence: 1401
Number of Binary non mergers with triple influence: 960


In [135]:
df.to_csv("Data/binary-merger-data.csv",index = False)
print("saved")

saved


In [139]:
m1_list = []
m1_id = []
m2_list = []
m2_id = []
m3_list = []
m3_id = []

In [140]:
Ntriple = np.size(bh1id1)

In [149]:
ix_remove = []

for i in range(Ntriple):

    m1 = Triple_df["M1_bin_1"].iloc[i]
    m2 = Triple_df["M2_bin_1"].iloc[i]
    
    qin_old = m2/m1

    mA = Triple_df["M1_bin_2"].iloc[i]
    m3 = Triple_df["M2_bin_2"].iloc[i]

    dM = mA-(m1+m2)

    if(dM>0):
        m1_new = m1 + dM*(1-qin_old)
        m2_new = m2 + dM*qin_old

        if (m1_new>m2_new):
            m1_list.append(m1_new)
            m1_id.append(Triple_df["bhid1_bin_1"].iloc[i])
            m2_list.append(m2_new)
            m2_id.append(Triple_df["bhid2_bin_1"].iloc[i])
        
        else:
            m1_list.append(m2_new)
            m1_id.append(Triple_df["bhid2_bin_1"].iloc[i])
            m2_list.append(m1_new)
            m2_id.append(Triple_df["bhid1_bin_1"].iloc[i])
        
        m3_list.append(m3)
        m3_id.append(Triple_df["bhid1_bin_2"].iloc[i])
    
    else:
        ix_remove.append(i)
        

m1_list = np.array(m1_list)
m2_list = np.array(m2_list)
m3_list = np.array(m3_list)

m1_id = np.array(m1_id)
m2_id = np.array(m2_id)
m3_id = np.array(m3_id)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [142]:
Triple_df.iloc[ix_remove]

,bhid1_bin_1,bhid2_bin_1,bhid1_bin_2,bhid2_bin_2,M1_bin_1,M2_bin_1,M1_bin_2,M2_bin_2,t1,t2,tmerger
497,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,5.608023e+07,5.077300e+07,5.764539e+07,2.525074e+07,2.102321e+10,9.854869e+09,8.888190e+09
498,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,1.467121e+08,1.188970e+08,1.659010e+08,1.266006e+08,1.947578e+10,1.201756e+10,1.052584e+10
500,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,9.016216e+07,8.101857e+07,1.061407e+08,6.139939e+06,1.120306e+11,2.790481e+10,1.131227e+10


In [143]:
Triple_df

,bhid1_bin_1,bhid2_bin_1,bhid1_bin_2,bhid2_bin_2,M1_bin_1,M2_bin_1,M1_bin_2,M2_bin_2,t1,t2,tmerger
0,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,3.286581e+07,1.253781e+07,5.276829e+07,8.204738e+06,2.055999e+09,1.988185e+09,1.985633e+09
1,9.223372e+18,9.223372e+18,3.402509e+18,9.223372e+18,1.059404e+07,2.885124e+06,3.132749e+08,4.211761e+07,3.255727e+11,7.265512e+09,5.529691e+09
2,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,1.655162e+07,6.537747e+06,7.257406e+07,4.653945e+07,5.274952e+10,1.032617e+10,3.025313e+09
3,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,1.259004e+07,3.439988e+06,2.386990e+07,2.216544e+06,5.883422e+09,1.284130e+12,1.972738e+09
4,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,1.325181e+08,1.408275e+07,4.551391e+08,1.002870e+06,1.206497e+10,7.389760e+09,4.814316e+09
...,...,...,...,...,...,...,...,...,...,...,...
526,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,5.557129e+07,8.642577e+06,1.019236e+08,6.895766e+07,3.514319e+10,1.323119e+10,1.241580e+10
527,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,2.426127e+08,2.139748e+07,2.724291e+08,1.692168e+06,1.468992e+10,1.466670e+10,1.354208e+10
528,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,1.325318e+07,8.303175e+06,2.158876e+07,6.823273e+06,1.927055e+11,1.358702e+10,1.311330e+10
529,9.223372e+18,9.223372e+18,9.223372e+18,9.223372e+18,4.575134e+07,9.468752e+06,5.555056e+07,5.338969e+07,4.607816e+10,1.945157e+10,1.355722e+10


In [144]:
M1 = m1_list
qin = m2_list/m1_list
qout = m3_list/(m1_list + m2_list)

t_triple = Triple_df["tmerger"].to_numpy()
z_triple = z_at_value(cosmo.age,(t_triple/10**9)*u.Gyr,zmin=1e-10)
z_triple = np.array(z_triple)

In [145]:
t_triple = np.delete(t_triple,ix_remove)
z_triple = np.delete(z_triple,ix_remove)

In [146]:
column_name = ['M1','M2','M3','qin','qout','t_triple','z_triple','M1_ID','M2_ID','M3_ID']
Triple_df_mod = pd.DataFrame({'M1': M1,'M2':m2_list,'M3':m3_list,'qin': qin, 'qout': qout, 't_triple' : t_triple, 'z_triple' : z_triple,'M1_ID':m1_id,'M2_ID':m2_id,'M3_ID':m3_id},columns=column_name)

In [147]:
Triple_df_mod

,M1,M2,M3,qin,qout,t_triple,z_triple,M1_ID,M2_ID,M3_ID
0,3.742097e+07,1.534732e+07,8.204738e+06,0.410126,0.155486,1.985633e+09,3.257459,9.223372e+18,9.223372e+18,9.223372e+18
1,2.287451e+08,8.452986e+07,4.211761e+07,0.369537,0.134443,5.529691e+09,1.101818,9.223372e+18,9.223372e+18,3.402509e+18
2,4.649029e+07,2.608377e+07,4.653945e+07,0.561058,0.641268,3.025313e+09,2.201414,9.223372e+18,9.223372e+18,9.223372e+18
3,1.828782e+07,5.582083e+06,2.216544e+06,0.305235,0.092859,1.972738e+09,3.276165,9.223372e+18,9.223372e+18,9.223372e+18
4,4.082678e+08,4.687125e+07,1.002870e+06,0.114805,0.002203,4.814316e+09,1.320180,9.223372e+18,9.223372e+18,9.223372e+18
...,...,...,...,...,...,...,...,...,...,...
523,8.741631e+07,1.450728e+07,6.895766e+07,0.165956,0.676562,1.241580e+10,0.103086,9.223372e+18,9.223372e+18,9.223372e+18
524,2.502891e+08,2.214000e+07,1.692168e+06,0.088458,0.006211,1.354208e+10,0.016267,9.223372e+18,9.223372e+18,9.223372e+18
525,1.326529e+07,8.323477e+06,6.823273e+06,0.627463,0.316057,1.311330e+10,0.048085,9.223372e+18,9.223372e+18,9.223372e+18
526,4.601341e+07,9.537147e+06,5.338969e+07,0.207269,0.961101,1.355722e+10,0.015170,9.223372e+18,9.223372e+18,9.223372e+18


In [148]:
Triple_df_mod.to_csv('Data/Triple-mass-redshift.csv',index = False)